In [1]:
import os

In [2]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\Chicken-Disease-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [16]:
from Chicken_Disease_Classifier.constants import *
from Chicken_Disease_Classifier.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config    

In [21]:
import os
import urllib.request as request
import zipfile
from Chicken_Disease_Classifier import logger
from Chicken_Disease_Classifier.utils.common import get_size

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)         



In [29]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-18 10:39:00,900: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-18 10:39:00,912: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-18 10:39:00,915: INFO: common: created directory at: artifacts]
[2023-06-18 10:39:00,917: INFO: common: created directory at: artifacts/data_ingestion]


[2023-06-18 10:39:18,692: INFO: 1157066443: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7ED0:607D:8200F:D5FB1:648E916C
Accept-Ranges: bytes
Date: Sun, 18 Jun 2023 05:09:04 GMT
Via: 1.1 varnish
X-Served-By: cache-del21738-DEL
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1687064943.672217,VS0,VE2166
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 8c967f048404f29b5cf93b2d3819d2ac8de5336a
Expires: Sun, 18 Jun 2023 05:14:04 GMT
Source-Age: 2

]
